## Business case
### Help stakeholder to make better financial decisions regarding selling and renovating houses with the potential for big investment returns according to data analysis in order to setup modeling for forecast ..use linearRegression 

## Import Cleaned data 

In [31]:
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import operator
# import to divide our data into train and test data
from sklearn.model_selection import train_test_split
# import to create polynomial features
from sklearn.preprocessing import PolynomialFeatures
# import of the linear regression model
from sklearn.linear_model import LinearRegression
# import of our evaluation metrics
from sklearn.metrics import mean_squared_error, r2_score

RSEED = 12

In [33]:
data_model=pd.read_csv('data/model_data.csv')
data_model.head()

,Unnamed: 0,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,...,lat,long,sqft_living15,sqft_lot15,sold_year,sold_month,sold_day,new_renovate,Age_house,new_id
0,0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,NaN,...,47.5112,-122.257,1340,5650,2014,10,0,0,61,1
1,1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0.0,...,47.7210,-122.319,1690,7639,2014,12,1,1,65,1
2,2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0.0,...,47.7379,-122.233,2720,8062,2015,2,2,2,83,1
3,3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0.0,...,47.5208,-122.393,1360,5000,2014,12,1,0,51,1
4,4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0.0,...,47.6168,-122.045,1800,7503,2015,2,2,0,29,1


## Split Data Training and Test

In [65]:
# Split data into train and test set
import sklearn.model_selection
X =data_model[['sqft_living','bedrooms']] ## as we have good correlation between price and sqft_living and price and bedrooms
y=data_model['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RSEED)

## Modeling 
### First model: linear regression

In [35]:
# Instantiate our model 

lin_reg = LinearRegression()
# Fit the model using our train data 
lin_reg.fit(X_train, y_train)

# Make predictions on the test data and save those in "y_pred"



LinearRegression()

## Prediction

In [36]:
y_pred_test = lin_reg.predict(X_test)
y_pred_train = lin_reg.predict(X_train)

## Evaluation Model & Error analysis 

In [37]:
from function import calculate_metrics
print("Linear Regression Model:")
print("---"*10)
calculate_metrics(y_train,y_pred_train, y_test, y_pred_test)  # The optimal RMSE is  Zero 

Linear Regression Model:
------------------------------
Metrics on training data
RMSE: 258760.848
R2: 0.508
------------------------------
Metrics on test data
RMSE: 255657.177
R2: 0.505
------------------------------


### RMSE value in traning data higher than RMSR value in test data that mean our model can handling the actual value better and its has good regularization .

In [66]:
import plotly.express as px
import numpy as np

# Assuming you have y_test and y_pred_test as your true and predicted data
# Calculate residuals
residuals = np.array(y_test) - np.array(y_pred_test)

# Create the scatter plot
fig = px.scatter(x=y_pred_test, y=residuals, title='Residual Values vs. Predicted Values',
                 labels={'x': 'Predicted Values', 'y': 'Residuals Values'})

# Add a horizontal line at y=0 for reference
fig.add_shape(type='line', x0=min(y_pred_test), x1=max(y_pred_test), y0=0, y1=0,
              line=dict(color='red', dash='dash'))
fig.update_layout(width=1200,height=600)

fig.show()


### The  pattern in residual plots are a sign for equilibrium prediction , as the accumulation of residual data point above zero and below zero no big different in model . ´ there some outlier in positive y_axis ´ that mean the prediction less that actual

In [67]:
import plotly.express as px
import numpy as np

# Assuming you have y_test and y_pred_test as your true and predicted data

# Create the scatter plot
fig = px.scatter(x=y_pred_test, y=y_test, title='True Values vs. Predicted Values',
                 labels={'x': 'Predicted Values', 'y': 'True Value'})

# Calculate the linear regression line
slope, intercept = np.polyfit(y_pred_test, y_test, 1)
regression_line = slope * y_pred_test + intercept

# Add the linear regression line to the plot
fig.add_scatter(x=y_pred_test, y=regression_line, mode='lines',
                line=dict(color='red', dash='dash'),
                name=f'Linear Regression Line (y={slope:.2f}x + {intercept:.2f})')
fig.update_layout(width=1200,height=600)
fig.show()



## linear regression with polynomial features to increase complexity to our model 

In [40]:
# here we will add the second degree polynomial features to increase complexity to our model 
polynomial_features = PolynomialFeatures(degree=2, include_bias=False)
x_poly_train = polynomial_features.fit_transform(X_train)
x_poly_test = polynomial_features.transform(X_test)

model_poly = LinearRegression()
model_poly.fit(x_poly_train, y_train)
y_pred_test = model_poly.predict(x_poly_test)
y_pred_train = model_poly.predict(x_poly_train)

## Evaluation Model & Error analysis 

In [63]:
print(" Linear Regression + Polynomial  (degree 2):")
print("---"*10)
calculate_metrics(y_train,y_pred_train, y_test, y_pred_test)

 Linear Regression + Polynomial  (degree 2):
------------------------------
Metrics on training data
RMSE: 249307.936
R2: 0.543
------------------------------
Metrics on test data
RMSE: 247495.89
R2: 0.536
------------------------------


## Tune Hyperparameter 1

#### Define model like  LinearRegression not work ,because hyperparameter search using RandomizedSearchCV to a LinearRegression model from scikit-learn, and some of the hyperparameters you're trying to optimize ('n_jobs', 'positive', etc.) are not applicable or valid for LinearRegression.
### So we can use Ridge or Lasso instead 


In [70]:

#from scipy.stats import loguniform

from sklearn.linear_model import Ridge
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import RandomizedSearchCV

# define model
model = Ridge()
# define evaluation
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define search space
space = dict()
space['solver'] = ['svd', 'cholesky', 'lsqr', 'sag']
space['alpha'] = loguniform(1e-5, 100)
space['fit_intercept'] = [True, False]
space['random_state'] = [True, False]
# define search
search = RandomizedSearchCV(model, space, n_iter=100, scoring='neg_mean_absolute_error', n_jobs=-1, cv=cv, random_state=1) 
# decrease value of parameter n_iter=500  from 500 to 100 make the process of search quick ,
# execute search
result = search.fit(X_train, y_train)
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

234513.49s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
234513.49s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
234513.49s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
234513.49s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
234513.49s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
234513.49s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
234513.49s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
234513.50s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfroz

Best Score: -170659.73450087168
Best Hyperparameters: {'alpha': 2.213755806263768e-05, 'fit_intercept': True, 'random_state': False, 'solver': 'cholesky'}


/Users/anas/.pyenv/versions/3.11.3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/anas/.pyenv/versions/3.11.3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/anas/.pyenv/versions/3.11.3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [71]:
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: -170659.73450087168
Best Hyperparameters: {'alpha': 2.213755806263768e-05, 'fit_intercept': True, 'random_state': False, 'solver': 'cholesky'}


## Instantiate Ridge Model 

In [72]:
# Instantiate our model with Hyperparameter

ridge_model = Ridge(alpha= 2.213755806263768e-05,solver='cholesky',fit_intercept=True,random_state=False)


## Training Model Ridge 

In [73]:

# Fit the model using our train data 
ridge_model.fit(X_train, y_train)

# Make predictions on the test data and save those in "y_pred"

y_pred_test = ridge_model.predict(X_test)
y_pred_train = ridge_model.predict(X_train)

## Evaluation Model & Error analysis 

In [75]:
print("results of Ridge model with Hyperparameter ")
print("---"*10)
calculate_metrics(y_train,y_pred_train, y_test, y_pred_test)

results of Ridge model with Hyperparameter 
------------------------------
Metrics on training data
RMSE: 258760.848
R2: 0.508
------------------------------
Metrics on test data
RMSE: 255657.177
R2: 0.505
------------------------------


## Notice : there are not clear develop after add Ridge model with hyperparameter to increase R_2

In [76]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_pred_test)  
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 65360592204.8709


## Develop the model by add new features  

In [48]:
# Add Age_house and  zipcode feature to make more complexity 
X =data_model[['sqft_living','bedrooms','Age_house','zipcode']] ## as we have good correlation between price and sqft_living and price and bedrooms
y=data_model['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RSEED)

## Instantiate LinearRegression Modeling 

In [49]:
# Instantiate our model 

lin_reg = LinearRegression()
# Fit the model using our train data 
lin_reg.fit(X_train, y_train)

# Make predictions on the test data and save those in "y_pred"

y_pred_test = lin_reg.predict(X_test)
y_pred_train = lin_reg.predict(X_train)

## Evaluation Model & Error analysis 

In [50]:
from function import calculate_metrics
print("Linear Regression Model:")
print("---"*10)
calculate_metrics(y_train,y_pred_train, y_test, y_pred_test)  # The optomal RMSE IS ZERO 

Linear Regression Model:
------------------------------
Metrics on training data
RMSE: 249307.519
R2: 0.543
------------------------------
Metrics on test data
RMSE: 247490.832
R2: 0.536
------------------------------


## linear regression with polynomial features to increase complexity to our model 

In [51]:
# here we will add the second degree polynomial features to increase complexity to our model 
polynomial_features = PolynomialFeatures(degree=3, include_bias=False)
x_poly_train = polynomial_features.fit_transform(X_train)
x_poly_test = polynomial_features.transform(X_test)

model_poly = LinearRegression()
model_poly.fit(x_poly_train, y_train)
y_pred_test = model_poly.predict(x_poly_test)
y_pred_train = model_poly.predict(x_poly_train)

## Evaluation Model & Error analysis 

In [52]:
print("Polynomial Regression Model (degree 3):")
print("---"*10)
calculate_metrics(y_train,y_pred_train, y_test, y_pred_test)

Polynomial Regression Model (degree 3):
------------------------------
Metrics on training data
RMSE: 233157.85
R2: 0.6
------------------------------
Metrics on test data
RMSE: 228000.009
R2: 0.606
------------------------------


## Tune Hyperparameter 2

In [53]:
from scipy.stats import loguniform

from sklearn.linear_model import Ridge
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import RandomizedSearchCV

# define model
model = Ridge()
# define evaluation
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define search space
space = dict()
space['solver'] = ['svd', 'cholesky', 'lsqr', 'sag']
space['alpha'] = loguniform(1e-5, 100)
space['fit_intercept'] = [True, False]
space['random_state'] = [True, False]
# define search
search = RandomizedSearchCV(model, space, n_iter=500, scoring='neg_mean_absolute_error', n_jobs=-1, cv=cv, random_state=1)
# execute search
result = search.fit(X_train, y_train)
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

/Users/anas/.pyenv/versions/3.11.3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/anas/.pyenv/versions/3.11.3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/anas/.pyenv/versions/3.11.3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/anas/.pyenv/versions/3.11.3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/anas/.pyenv/versions/3.11.3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/User

Best Score: -163367.14300361098
Best Hyperparameters: {'alpha': 96.51887517503673, 'fit_intercept': True, 'random_state': False, 'solver': 'lsqr'}


/Users/anas/.pyenv/versions/3.11.3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [54]:
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: -163367.14300361098
Best Hyperparameters: {'alpha': 96.51887517503673, 'fit_intercept': True, 'random_state': False, 'solver': 'lsqr'}


## Instantiate Ridge Model 

In [55]:
# Instantiate our model with Hyperparameter

ridge_model = Ridge(alpha=96.51887517503673,solver='lsqr',fit_intercept=True,random_state=False)

## Training Model 

In [56]:

# Fit the model using our train data 
ridge_model.fit(X_train, y_train)


Ridge(alpha=96.51887517503673, random_state=False, solver='lsqr')

## Prediction 

In [57]:
# Make predictions on the test data and save those in "y_pred"
y_pred_test = ridge_model.predict(X_test)
y_pred_train = ridge_model.predict(X_train)

## Evaluation Model & Error analysis 

In [58]:
print("results of Ridge model with Hyperparameter ")
print("---"*10)
calculate_metrics(y_train,y_pred_train, y_test, y_pred_test)

results of Ridge model with Hyperparameter 
------------------------------
Metrics on training data
RMSE: 249307.936
R2: 0.543
------------------------------
Metrics on test data
RMSE: 247495.89
R2: 0.536
------------------------------


In [59]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred_test)  
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 61254215473.26123


In [77]:
import plotly.express as px
import numpy as np

# Assuming you have y_test and y_pred_test as your true and predicted data
# Calculate residuals
residuals = np.array(y_test) - np.array(y_pred_test)

# Create the scatter plot
fig = px.scatter(x=y_pred_test, y=residuals, title='Residual Values vs. Predicted Values',
                 labels={'x': 'Predicted Values', 'y': 'Residuals'})

# Add a horizontal line at y=0 for reference
fig.add_shape(type='line', x0=min(y_pred_test), x1=max(y_pred_test), y0=0, y1=0,
              line=dict(color='red', dash='dash'))
fig.update_layout(width=1200,height=600)

fig.show()

In [78]:
import plotly.express as px
import numpy as np

# Assuming you have y_test and y_pred_test as your true and predicted data

# Create the scatter plot
fig = px.scatter(x=y_pred_test, y=y_test, title='True Values vs. Predicted Values',
                 labels={'x': 'Predicted Values', 'y': 'True Value'})

# Calculate the linear regression line
slope, intercept = np.polyfit(y_pred_test, y_test, 1)
regression_line = slope * y_pred_test + intercept

# Add the linear regression line to the plot
fig.add_scatter(x=y_pred_test, y=regression_line, mode='lines',
                line=dict(color='red', dash='dash'),
                name=f'Linear Regression Line (y={slope:.2f}x + {intercept:.2f})')
fig.update_layout(width=1200,height=600)
fig.show()